# Fitting a raidal profile to tropical cyclone observations

This notebook enables users to load an analysis track file and interactively fit a radial profile to the range of wind radii provided in the track file. The aim is to improve the quality of the simulated wind field by improving the fit of parameters in the profile (namely the Holland $\beta$ value and the environmental pressure).

At a minimum, it requires estimates of the central pressure and the radius to maximum winds. In the absence of pressure at the outermost closed isobar, the daily long-term-mean sea level pressure at the location of the storm is used. 



## Preliminaries

Set up the notebook with the required modules for plotting, analysis and the fiddly things like file loaders. 

In [11]:
#%matplotlib inline

from __future__ import print_function # py 2.7 compat.

import sys
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt

Use [``matplotlib``](http://matplotlib.org/index.html) for plotting the data up. We use [``basemap``](http://matplotlib.org/basemap/) to plot the track on a simple map.

In [21]:
from matplotlib.collections import LineCollection
from matplotlib.colors import Normalize, BoundaryNorm, ListedColormap
from matplotlib.cm import get_cmap
from matplotlib.dates import HourLocator, DateFormatter
from matplotlib.ticker import MultipleLocator

from mpl_toolkits.basemap import Basemap
from mpl_toolkits.basemap import cm as basemapcm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn; seaborn.set()
from datetime import datetime, timedelta

# Import widgets for interactive notebook
from IPython.html.widgets import interact, fixed
from IPython.html import widgets

from Utilities.metutils import convert
from Utilities.track import Track
from PlotInterface.tracks import TrackMapFigure

The ``NotebookFinder`` function allows me to import other IPython notebooks as Python modules. I use another notebook to set up the interactive widget for loading a file.

In [22]:
from NoteBookFinder import NotebookFinder
sys.meta_path.append(NotebookFinder())

We build a widget to allow selection of a track file. This is from [Interactive Widgets](http://nbviewer.ipython.org/github/adrn/ipython/blob/2.x/examples/Interactive%20Widgets/File%20Upload%20Widget.ipynb).

In [23]:
import FileWidgets

FileLoader = FileWidgets.FileLoaderWidget()
FileLoader

We use the existing wind modules from TCRM to generate the profile. These are part of the ``wind.windmodels`` module. For comparison, we use the Powell and Willoughby profiles as well. The former calculates $\beta$ as a function of maximum wind speed and latitude, while the latter uses maximum wind speed only.

The track data needs to be loaded from a csv file. Here's the challenge - different track files have different formats, with different field names for the same data. Some files have fields that others don't, while they all have different date formats and potentially different units for data like pressure and wind speed. 

For initial prototyping, I'm using the format of the BoM's Analysis Track data for TC Marcia, provided by David Grant (Qld Regional Office Severe Weather Section). The intensity and track data is preliminary, based on operational estimates and subject to change following post analysis.

In [24]:
COLNAMES = ['Datetime', 'Latitude', 'Longitude', 'Symbol', 'Category','CentralPressure', 'PressureOCI', 'RadiusOCI', 'Radius1000hPa',
            'RadiusMaxWinds', 'MeanWind', 'WindSpeed', 'VerticalExtent', 'Uncertainty', 'FinalT', 'CurrentIntensity',
            'R34', 'R41', 'R48', 'R64', 'NER34', 'SER34', 'SWR34', 'NWR34', 'NER41', 'SER41', 'SWR41', 
            'NWR41', 'NER48', 'SER48', 'SWR48', 'NWR48', 'NER64', 'SER64', 'SWR64', 'NWR64']
COLTYPES = [datetime, 'f8', 'f8', 'i', 'i', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'i', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 
            'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f']
COLUNITS = ('', 'degree', 'degree', '', '', 'hPa', 'hPa', 'nm', 'nm',
            'nm', 'kts', 'kts', '', 'nm', '', '', 
            'nm', 'nm', 'nm', 'nm', 'nm', 'nm', 'nm', 'nm', 
            'nm', 'nm', 'nm', 'nm', 'nm', 'nm', 'nm', 'nm', 
            'nm', 'nm', 'nm', 'nm')
DATEFORMAT = "%Y-%m-%dT%H:%M:%SZ"
dtype = np.dtype({'names':COLNAMES, 'formats':COLTYPES})
converters = {
    0: lambda s: datetime.strptime(s.strip(), DATEFORMAT),
    5: lambda s: convert(float(s.strip() or 0), COLUNITS[5], 'Pa'),
    6: lambda s: convert(float(s.strip() or 0), COLUNITS[6], 'Pa'),
    7: lambda s: convert(float(s.strip() or 0), COLUNITS[7], 'km'),
    8: lambda s: convert(float(s.strip() or 0), COLUNITS[8], 'km'),
    9: lambda s: convert(float(s.strip() or 0), COLUNITS[9], 'km'),
    10: lambda s: convert(float(s.strip()), COLUNITS[10], 'mps'),
    11: lambda s: convert(float(s.strip()), COLUNITS[11], 'mps'),
    13: lambda s: convert(float(s.strip()), COLUNITS[13], 'km'),
    16: lambda s: convert(float(s.strip()), COLUNITS[16], 'km'),
    16: lambda s: convert(float(s.strip()), COLUNITS[16], 'km'),
    17: lambda s: convert(float(s.strip()), COLUNITS[17], 'km'),
    18: lambda s: convert(float(s.strip()), COLUNITS[18], 'km'),
    19: lambda s: convert(float(s.strip()), COLUNITS[19], 'km'),
    20: lambda s: convert(float(s.strip()), COLUNITS[20], 'km'),
    21: lambda s: convert(float(s.strip()), COLUNITS[21], 'km'),
    22: lambda s: convert(float(s.strip()), COLUNITS[22], 'km'),
    23: lambda s: convert(float(s.strip()), COLUNITS[23], 'km'),
    24: lambda s: convert(float(s.strip()), COLUNITS[24], 'km'),
    25: lambda s: convert(float(s.strip()), COLUNITS[25], 'km'),
    26: lambda s: convert(float(s.strip()), COLUNITS[26], 'km'),
    27: lambda s: convert(float(s.strip()), COLUNITS[27], 'km'),
    28: lambda s: convert(float(s.strip()), COLUNITS[28], 'km'),
    29: lambda s: convert(float(s.strip()), COLUNITS[29], 'km'),
    30: lambda s: convert(float(s.strip()), COLUNITS[30], 'km'),
    31: lambda s: convert(float(s.strip()), COLUNITS[31], 'km'),
    32: lambda s: convert(float(s.strip()), COLUNITS[32], 'km'),
    33: lambda s: convert(float(s.strip()), COLUNITS[33], 'km'),
    34: lambda s: convert(float(s.strip()), COLUNITS[34], 'km')
}
delimiter = ','
skip_header = 1
usecols = tuple(range(36))
missing_value = "NaN"
filling_values = 0
filename = FileLoader.filename

data = np.genfromtxt(filename, dtype, delimiter=delimiter, skip_header=skip_header, converters=converters,
                     missing_values=missing_value, filling_values=filling_values, usecols=usecols, autostrip=True)

times = data['Datetime']
timelist = [d.strftime('%Y-%m-%d %H:%M') for d in data['Datetime']]

diffs = [d1 - d0 for d0, d1 in zip(times[:-1], times[1:])]
dt =  np.array([0.0] + [round(d.days * 24. + d.seconds / 3600.)
                             for d in diffs], 'f')
index=np.zeros(len(times), 'i')
from Utilities.loadData import getSpeedBearing

speed, bearing = getSpeedBearing(index, data['Longitude'], data['Latitude'], dt)
speed = convert(speed, 'kmh', 'mps')
age = np.cumsum(dt)
lon = data['Longitude']
lat = data['Latitude']
penv = data['PressureOCI']
pcentre = data['CentralPressure']
rmax = data['RadiusMaxWinds']

The data is now stored as a ``numpy.recarray`` object, which makes it easier to work with. 

Check the date range:

In [25]:
print("First time: {0}".format(timelist[0]))
print("Final time: {0}".format(timelist[-1]))

Plot the track on a map to give some context. These two functions enable you to colourize the line segments (e.g. based on intensity). The classification is based on the estimated maximum gust wind speed and uses the Australian TC intnsity scale.

In [26]:
def make_segments(x, y):
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    return segments

def colorline(x, y, z=None, linewidth=1.0, alpha=1.0):
    if z is None:
        z = np.linspace(0.0, 1.0, len(x))

    if not hasattr(z, '__iter__'):
        z = np.array([z])

    z = np.asarray(z)

    segments = make_segments(x, y)
    cmap = ListedColormap(['0.75', '#0FABF6', '#0000FF',
                            '#00FF00', '#FF8100', '#ff0000'])
    norm = BoundaryNorm([0, 17.5, 24.5, 32.5, 44.2, 55.5, 1000.], cmap.N)
    lc = LineCollection(segments, array=z, cmap=cmap, 
                        norm=norm, linewidth=linewidth, alpha=alpha)
    
    ax = plt.gca()
    ax.add_collection(lc)

The map is set up to cover the extent of the track, with a 2$\^{circ}$ margin around it. I haven't added parallels and meridians at this time (but could be a good exercise to do). 

In [27]:
minLon = np.floor(lon.min()) - 2
maxLon = np.ceil(lon.max()) + 2
minLat = np.floor(lat.min()) - 2
maxLat = np.ceil(lat.max()) + 2

xx = np.arange(minLon, maxLon + 0.1, 0.1)
yy = np.arange(minLat, maxLat + 0.1, 0.1)
xgrid, ygrid = np.meshgrid(xx, yy)

m = Basemap(projection='mill', llcrnrlon=minLon, llcrnrlat=minLat, 
            urcrnrlon=maxLon, urcrnrlat=maxLat, resolution='h')
m.drawcoastlines()
m.drawstates()
m.drawcountries()
m.fillcontinents('coral')
x,y = m(lon, lat)
colorline(x,y,z=data['WindSpeed'])
plt.show()

Define a simple function to calculate $\beta$ using the method of Powell *et al.* (2005).

In [28]:
def powell_beta(lat, rmax):
    rm = ma.array(rmax, mask=np.isnan(rmax))
    beta = 1.881093 - 0.010917 * np.abs(lat) - 0.005567 * rm
    return beta

Plot the main parameters over the lifetime of the event. This can help identify the times where sufficient parameters are available to do some sort of profile fitting. 

In [29]:
def plottimes(data, beta=None):
    dayLocator = DayLocator()
    hourLocator = HourLocator(interval=6)
    dateFormat = DateFormatter('%H:%MZ\n%Y-%m-%d')
    times = data['Datetime']
    fig = plt.figure(figsize=(16,10))
    ax1 = plt.subplot(2,1,1)
    ax1.plot(times, convert(data['CentralPressure'], "Pa", "hPa"), linestyle='-', 
             color='k', linewidth=2, label="Central pressure (hPa)")
    ax1.scatter(times, convert(data['CentralPressure'], "Pa", "hPa"), marker='+', color='k' )

    ax1.plot(times, convert(data['PressureOCI'], "Pa", "hPa"), linestyle='-', 
             color='0.75', linewidth=2, label="Environmental pressure (hPa)")
    ax1.scatter(times, convert(data['PressureOCI'], "Pa", "hPa"), marker='+', color='0.75' )

    ax1.set_ylabel("Pressure (hPa)")
    ax1.set_xlabel("Date")
    ax1.xaxis.set_major_locator(dayLocator)
    ax1.xaxis.set_minor_locator(hourLocator)
    l = ax1.legend(loc='lower left')
    for t in l.get_texts():
        t.set_fontsize('x-small')
    ax1.grid(axis='x')

    ax2 = ax1.twinx()
    p = ax2.plot(times, data['MeanWind'], linestyle='-', color='r', 
                 linewidth=2, label="Mean wind speed (m/s)")
    ax2.scatter(times, data['MeanWind'], marker='+', color='r')
    
    ax2.plot(times, data['WindSpeed'], linestyle='-', color='pink', 
             linewidth=2, label="Maximum wind speed (m/s)")
    ax2.scatter(times, data['WindSpeed'], marker='+', color='pink')
    
    ax2.plot(times, data['WindSpeed'] - 0.7*speed, linestyle=':', color='r',
             linewidth=1, label='Modified maximum wind speed')
    
    ax2.plot(times,data['RadiusMaxWinds'], linestyle='-', color='b', 
             linewidth=2, label="Radius to maximum wind (km)")

    ax2.scatter(times,data['RadiusMaxWinds'], marker='+', color='b')
    ax2.set_ylabel("Wind speed (m/s)\nRadius (km)")
    ax2.xaxis.set_major_locator(dayLocator)
    ax2.xaxis.set_minor_locator(hourLocator)
    ax2.xaxis.set_major_formatter(dateFormat)
    plt.xlim((times[0], times[-1]))
    l = ax2.legend(loc='lower right')
    for t in l.get_texts():
        t.set_fontsize('x-small')
    plt.grid(axis='both')
    
    if beta is not None:
        pbeta = powell_beta(data['Latitude'], data['RadiusMaxWinds'])
        #print(len(pbeta))
        #print(len(times))
        ax3 = plt.subplot(2,1,2)
        ax3.plot(times, beta, linestyle='-', color='k', linewidth=2, label=r'$\beta$')
        ax3.scatter(times, beta, marker='+', color='k')
        ax3.plot(times, pbeta, linestyle='-', color='0.75', linewidth=2, label=r"Powell's $\beta$")
        ax3.scatter(times, pbeta, marker='+', color='0.75')
        ax3.set_ylabel(r'$\beta$')
        ax3.set_xlabel("Date")
        ax3.xaxis.set_major_locator(dayLocator)
        ax3.xaxis.set_minor_locator(hourLocator)
        ax3.xaxis.set_major_formatter(dateFormat)
        l = ax3.legend(loc='upper left')
        for t in l.get_texts():
            t.set_fontsize('x-small')
        ax3.grid()
        plt.xlim((times[0], times[-1]))
        
    plt.show()

        
plottimes(data)

The modified maximum wind speed is reduced by 0.7 times the forward speed of the cyclone. This is to account for the wavenumber-1 asymmetry induced in the vortex flow by the forward motion. In reality, it is likely not a constant value, but some boundary layer models use this simple approximation. 

Now to the task at hand. We're trying to fit the Holland profile as best as possible to the recorded wind radii. We perform this at each time step individually.

In [30]:
try:
    import wind.windmodels as windmodels
except ImportError:
    print("Cannot import windmodels. Check that the TCRM folder is in the PYTHONPATH")
    
holland = windmodels.HollandWindProfile
powell = windmodels.PowellWindProfile
willoughby = windmodels.WilloughbyWindProfile

In [31]:
def get_wind(profile, r):
    try:
        V = profile.velocity(r)
    except AssertionError:
        print("Missing required fields")
        return np.zeros(len(r))
    else:
        return V

Set up functions to caculate the residuals of the model fit compared to the analysis. 

In [32]:
def residuals(beta, r, v, rmax, vmax, lat, lon, penv, pcentre):
    """
    Return the residuals between the model fit and the analysis
    """
    #assert len(r)==len(v)
    robs = np.zeros(len(r)+1)
    vobs = np.zeros(len(v)+1)
    robs[1:] = r
    robs[0] = rmax
    vobs[1:] = v
    vobs[0] = vmax
    h = holland(lat, lon, penv, pcentre, rmax, beta)
    try:
        vmodel = h.velocity(robs)
    except AssertionError:
        return np.zeros(len(robs))
    
    err = abs(vmodel) - vobs
    return err

from scipy.optimize import leastsq

def minimise(r, v, rmax, vmax, lat, lon, penv, pcentre, beta=1.5):
    """
    Provide a functional interface to the leastsq function from scipy 
    and return the parameter value.
    
    """
    plsq = leastsq(residuals, [beta], args=(r, v, rmax, vmax, lat, lon, penv, pcentre))
    return plsq[0]



The second panel here plots the optimum $\beta$ value from Holland's profile (see Holland, 1980), estimated by fitting the profile to the analysed radii to various wind speeds (gales, strong gale, storm and hurricane-force wind speeds), as well as the radius to maximum winds.  We use least-squares minimisation to obtain the $\beta$ value. 

In [33]:
beta = 1.8*np.ones(len(times))


for i, t in enumerate(times):
    pc = pcentre[i]
    pe = penv[i]
    rm = rmax[i]
    roci = data['RadiusOCI'][i]
    vm = data['WindSpeed'][i] - 0.7*speed[i]
    loni = lon[i]
    lati = lat[i]

    if np.isnan(data['R34'][i]):
        r34 = np.max([data['NER34'][i], data['NWR34'][i],
                       data['SWR34'][i], data['SER34'][i]])
    else:
        r34 = data['R34'][i]
        
    if np.isnan(data['R41'][i]):
        r41 = np.max([data['NER41'][i], data['NWR41'][i],
                       data['SWR41'][i], data['SER41'][i]])
    else:
        r41 = data['R41'][i]    
    
    r48 = data['R48'][i]
    r64 = data['R64'][i]
    
    r = ma.array([r64, r48, r41, r34],
                 mask=np.isnan([r64, r48, r41, r34]))
    v = ma.array([64*1.4, 48*1.4, 41*1.4, 34*1.4],
                 mask=np.isnan([r64, r48, r41, r34]))
    
    if np.isnan(rm) or np.isnan(pe) or np.isnan(pc) or np.isnan(vm):
        beta[i] = beta[i]
    else:
        beta[i] = minimise(r, v, rm, vm, lati, loni, pe, pc)[0]


plottimes(data, beta)

Save the data to a formatted file, including the $\beta$ values, for use in simulating the TC using TCRM. Note that you will have to manually edit the file to amend any ``nan`` values, either by removing the records containing them, or replacing them with estimated values.

In [34]:
track = np.core.records.fromarrays([np.ones(len(times)), timelist, age, lon, lat, speed, bearing, pcentre/100., penv/100., rmax, beta ],
                 dtype=np.dtype({'names':("Index", "Datetime", "TimeElapsed", "Longitude", "Latitude", "Speed", "Bearing", 
                                          "CentralPressure", "EnvPressure", "Rmax", "beta"),
                                'formats':('i', 'object','f', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8')}
                                )
                 ).T

header = 'CycloneNumber,Datetime,TimeElapsed,Longitude,' + \
                 'Latitude,Speed,Bearing,' + \
                 'CentralPressure,EnvPressure,rMax,beta\n'
        
fmt = '%i,%s,%7.3f,%8.3f,%8.3f,%6.2f,%6.2f,%7.2f,%7.2f,%6.2f,%6.3f'.split(',')
with open("C:/temp/track.csv", 'w') as fp:
            fp.write('%' + header)
            if len(track) > 0:
                np.savetxt(fp, track, fmt=fmt,delimiter=',')        

Here, we can interact with the data for a chosen time step. Select a time from the drop down list, then move the slider to visually find a fit to the data points. 

This highlights one issue with the analysis. There are some time steps where the radius to hurricane-force winds ($R_H$) is equal to the radius of maximum winds ($R_{max}$), but the maximum winds are greater than hurricane force. This is suspected to be because the radius to hurricane-force winds is only analysed when hurricane-force winds extend _around_ the cyclone, but the maximum winds are at a single location near the cyclone centre, and usually arises because of the asymmetry induced in the wind field due to the forward motion of the storm. 

To (partially) account for this, we presume that the forward speed of the storm contributes towards the maximum wind speed. Based on results from previous studies, we reduce the maximum wind speed ($V_m$) by $0.7 \times V_{forward}$. 

In [37]:
def fitprofile(timestamp, beta=1.5):
    
    
    idx = timelist.index(timestamp)
    
    pc = data['CentralPressure'][idx]
    pe = data['PressureOCI'][idx]
    rm = data['RadiusMaxWinds'][idx]
    roci = data['RadiusOCI'][idx]
    vm = data['WindSpeed'][idx] - 0.7*speed[idx]

    if np.isnan(data['R34'][idx]):
        r34 = np.mean([data['NER34'][idx], data['NWR34'][idx],
                       data['SWR34'][idx], data['SER34'][idx]])
    else:
        r34 = data['R34'][idx]
        
    if np.isnan(data['R41'][idx]):
        r41 = np.mean([data['NER41'][idx], data['NWR41'][idx],
                       data['SWR41'][idx], data['SER41'][idx]])
    else:
        r41 = data['R41'][idx]    
    
    r48 = data['R48'][idx]
    r64 = data['R64'][idx]
    
    r = np.arange(0, 201, 1., dtype=float)
    h = holland(lat[idx], lon[idx], pe, pc, rm, beta)
    p = powell(lat[idx], lon[idx], pe, pc, rm)
    w = willoughby(lat[idx], lon[idx], pe, pc, rm)
    try:
        V = h.velocity(r)
    except AssertionError:
        print("Missing required fields")
    else:
        fig = plt.figure(figsize=(16,12))
        ax = plt.subplot(1,1,1)
        ax.plot(r, abs(V), linewidth=2, color='b', label="Holland")
        Vp = p.velocity(r)
        Vw = w.velocity(r)
        ax.plot(r, abs(Vp), linewidth=1, color='r', label="Powell")
        ax.plot(r, abs(Vw), linewidth=1, color='g', label="Willoughby")

        ax.set_xlabel("Radial distance (km)")
        ax.set_ylabel("Wind speed (m/s)")
        ax.scatter(r34, convert(34*1.4, 'kts', 'mps'), s=30, marker='o')
        ax.scatter(r41, convert(41*1.4, 'kts', 'mps'), s=30, marker='o')
        ax.scatter(r48, convert(48*1.4, 'kts', 'mps'), s=30, marker='o')
        ax.scatter(r64, convert(64*1.4, 'kts', 'mps'), s=30, marker='o')
        ax.scatter(rm, vm - 0.7*speed[idx], s=30, marker='o', color='k')
        ax.set_xlim((0,200))
        ax.set_ylim((0, 80))
        textstr = 'vmax = %.1f m/s \nrmax = %.1f km\nProfile vmax = %.1f m/s'%(vm,rm,abs(V).max())
        props = dict(boxstyle='round', facecolor='gray', alpha=0.5)
        ax.text(0.85, 0.95, textstr, transform=ax.transAxes, fontsize=12, verticalalignment='top', bbox=props)
        ax.grid(True)
        ax.legend(loc=7)
        plt.show()

Select a time that has all the required fields. If there aren't sufficient fields, then a little message will be printed, but doesn't indicate which fields are missing. You will need to look through the input file to check which fields are missing, or to identify a time that has the required fields. 

Use the dropdown to select a different time stamp to plot up. Once you have sufficient data to generate a plot, use the slider to adjust the beta value to visually get a good fit to the points. 

In [38]:
interact(fitprofile, 
         timestamp=widgets.DropdownWidget(values=timelist, value=timelist[0]), 
         beta=widgets.FloatSliderWidget(min=0.5, max=3.0, step=0.05, value=1.5, description="beta"))

### References

1. Holland, G. J. (1980): _An Analytic Model of the Wind and Pressure Profiles in Hurricanes_. Monthly Weather Review, __108__, 1212-1218
2. Powell, M., G. Soukup, S. Cocke, S. Gulati, N. Morisseau-Leroy, S. Hamid, N. Dorst, & L. Axe (2005):  _State of Florida hurricane loss projection model: Atmospheric science component_. Journal of Wind Engineering and Industrial Aerodynamics, __93__, 651-674.
